### Final Project

1. orientation
2. velocity

set body length to 1 unit, pool size about 1000x body size?

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import random as rng
from matplotlib import colors

Simulation code

In [6]:
#initialize coordinates and orientation 
N = 1000
poolsize = 1000

motcoords = [(rng.randint(0,poolsize),rng.randint(0,poolsize)) for i in np.arange(N)]

xcoords, ycoords = zip(*motcoords)


[(188, 83),
 (518, 260),
 (401, 193),
 (927, 795),
 (531, 382),
 (237, 909),
 (639, 881),
 (79, 786),
 (3, 707),
 (641, 812),
 (0, 397),
 (366, 982),
 (48, 523),
 (560, 676),
 (537, 961),
 (763, 539),
 (827, 752),
 (559, 504),
 (775, 957),
 (968, 842),
 (79, 2),
 (743, 334),
 (604, 255),
 (244, 528),
 (125, 846),
 (318, 73),
 (979, 130),
 (308, 414),
 (819, 428),
 (396, 754),
 (437, 82),
 (781, 826),
 (143, 992),
 (660, 82),
 (640, 740),
 (18, 300),
 (348, 878),
 (767, 350),
 (290, 956),
 (828, 886),
 (541, 139),
 (160, 489),
 (113, 88),
 (943, 660),
 (212, 131),
 (340, 142),
 (436, 909),
 (563, 793),
 (780, 701),
 (420, 42),
 (631, 103),
 (302, 773),
 (461, 815),
 (95, 347),
 (293, 577),
 (764, 487),
 (334, 42),
 (95, 827),
 (28, 464),
 (304, 107),
 (361, 404),
 (51, 644),
 (35, 34),
 (539, 744),
 (35, 106),
 (24, 333),
 (651, 497),
 (252, 487),
 (182, 316),
 (385, 469),
 (621, 941),
 (5, 473),
 (28, 974),
 (846, 554),
 (972, 464),
 (220, 48),
 (160, 221),
 (665, 285),
 (886, 845),
 (

Analysis